http://www.ibama.gov.br/manchasdeoleo-localidades-atingidas

In [1]:
import pandas as pd


url = "http://www.ibama.gov.br/phocadownload/emergenciasambientais/2019/manchasdeoleo/2019-12-02_LOCALIDADES_AFETADAS.xlsx"
df = pd.read_excel(
    url,
    parse_dates=["Data_Avist", "Data_Revis"],
)

df.dropna(inplace=True)
df.head()

,geocodigo,localidade,loc_id,municipio,estado,sigla_uf,Data_Avist,Data_Revis,Status,Latitude,Longitude
0,3201605,Praia de Guriri,3201605_43,Conceição da Barra,Espírito Santo,ES,2019-11-23,2019-11-23,Oleada - Vestigios / Esparsos,"18° 40' 43.70"" S","39° 44' 33.69"" W"
1,3201605,Praia de Guriri,3201605_46,Conceição da Barra,Espírito Santo,ES,2019-11-18,2019-11-19,Oleada - Vestigios / Esparsos,"18° 42' 14.87"" S","39° 44' 45.92"" W"
2,3201605,Praia de Guriri,3201605_45,Conceição da Barra,Espírito Santo,ES,2019-11-21,2019-11-27,Oleada - Vestigios / Esparsos,"18° 41' 54.57"" S","39° 44' 40.31"" W"
3,2207702,Praia do Pontal,2207702_4,Parnaíba,Piauí,PI,2019-11-08,2019-11-17,Oleada - Vestigios / Esparsos,"2° 45' 46.58"" S","41° 46' 54.03"" W"
4,3202405,Praia do Morro,3202405_39,Guarapari,Espírito Santo,ES,2019-11-28,2019-11-28,Oleada - Vestigios / Esparsos,"20° 39' 18.19"" S","40° 28' 42.74"" W"


In [2]:
import janitor


df.columns = df.columns.str.lower()

df = df.clean_names() \
  .remove_empty() \
  .rename_column("data_avist", "data avistagem") \
  .rename_column("data_revis", "data re-visita") \
  .rename_column("latitude", "lat") \
  .rename_column("longitude", "lon") \
  .encode_categorical(["status"])

In [3]:
df["municipio"] = df["municipio"] + ", " + df["estado"]
df.drop("estado", axis=1, inplace=True)

In [4]:
df["dias"] = (df["data re-visita"] - df["data avistagem"]).dt.days

df["data avistagem"] = df["data avistagem"].dt.strftime("%Y-%m-%d").str.replace("NaT", "na")
df["data re-visita"] = df["data re-visita"].dt.strftime("%Y-%m-%d").str.replace("NaT", "na")
df["dias"] = df["dias"].astype('Int64').astype(str).str.replace("nan", "na")

df["visita"] = (
    df["data avistagem"]
    + " / "
    + df["data re-visita"]
    + " (" + df["dias"] + " dias)"
)

df.drop(["data avistagem", "data re-visita", "dias"], axis=1, inplace=True)

Convert text position to degrees decimal float type.

In [5]:
def fix_pos(pos):
    if isinstance(pos, float):
        return pos
    deg, rest = pos.split("°")
    mi, rest = rest.strip().split("'")
    sec, hem = rest.strip().split('"')
    hem = hem.strip()

    deg, mi, sec = map(float, (deg, mi, sec))
    if hem in ["S", "W"]:
        sign = -1
    elif hem in ["N", "E"]:
        sign = +1
    else:
        raise ValueError(f"Unrecognized sign {sign}, expected 'S', 'W', 'E', or 'N'")

    return sign * (deg + mi/60 + sec/60/60)

In [6]:
df["lat"] = df["lat"].apply(fix_pos)
df["lon"] = df["lon"].apply(fix_pos)

Map it!

In [7]:
import folium
from folium.plugins import Fullscreen

import pandas as pd


m = folium.Map()
Fullscreen().add_to(m)

for k, row in df.iterrows():
    if "Nao Observado" in row["status"]:
        icon = folium.Icon(
            color='lightgray',
            icon='tint',
        )
    elif "Vestigios" in row["status"]:
        icon = folium.Icon(
            color='orange',
            icon='tint',
        )
    elif "Manchas" in row["status"]:
        icon = folium.Icon(
            color='red',
            icon='tint',
        )
    else:
        print(f"Unrecognized status {row['status']}.")

    location = row["lat"], row["lon"]
    table = pd.DataFrame(row).drop(["lon", "lat"]).to_html()
    folium.Marker(
        location=location,
        popup=table,
        icon=icon
    ).add_to(m)

m.fit_bounds(m.get_bounds())

In [8]:
m